<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/09%20-%20Word%20embedding%20e%20Word2Vec/glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GloVe
Il modello **GloVe (Global Vectors for Word Representation)** è un metodo non supervisionato per l'apprendimento della rappresentazione vettoriale delle parole. La prima versione di GloVe è stata creata nei laboratori di NLP di Standford nel 2014, i quali hanno messo a disposizione il modello pre-addestrato sull'intero corpus di testo di Wikipedia, con qualcosa come 6 miliardi di parole. In questo notebook utilizzeremo il modello pre-addestrato per creare lo strato di embedding di una rete neurale per classificare la polarità di recensioni di film utilizzando l'IMDB Movie Reviews Dataset.

## Prepariamo i dati

In precedenza abbiamo visto come scaricare e preprocessare l'IMDB Movie Reviews Dataset, Keras mette a disposizione tale dataset già preprocessato.
<br>**ATTENZIONE**<br>
Se caricando il dataset ottieni questo errore:<br>
*ValueError: Object arrays cannot be loaded when allow_pickle=False*
<br>
questo è casuato da un bug nell'ultima versione di keras, per correggerlo esegui il downgrade di numpy usando la cella di codice qui sotto e riavvia il kernel (su Colaboratory seleziona Runtime dalla barra dei comandi e clicca su Restart Runtime).

In [3]:
!pip install numpy==1.16.2

     |████████████████████████████████| 17.3MB 8.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.16.3
    Uninstalling numpy-1.16.3:
      Successfully uninstalled numpy-1.16.3


In [35]:
import numpy as np
from keras.datasets import imdb

(X_train, y_train), (X_test, y_test) = imdb.load_data()

print(X_train[0][:10])
print(y_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]
1


Ogni riga della lista con le features corrisponde ad una frase, ogni colonna contiene l'indice di una parola all'interno del vocabolario dell'intero corpus di testo. Il vettore con il target contiene un unico valore che può essere 0 per una recensione negativa o 1 per una recensione positiva.<br><br>
Per rendere le features un buon input per il nostro modello dobbiamo fare in modo che ogni frase abbia la stessa lunghezza, per farlo possiamo usare la funzione *pad_sequences(text)* di keras, che riduce tutte le frasi ad una lunghezza prefissata troncando quelle troppo lunghe e aggiungendo degli zeri a quelle troppo brevi. Usiamo una lunghezza comune di 50 parole.

In [47]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 50

X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

X_train.shape

(25000, 50)

Adesso i dati sono pronti.

## Carichiamo il modello GloVe

Possiamo scaricare il modello pre-addestrato [da questo link](http://nlp.stanford.edu/data/glove.6B.zip). Se utilizzi Google Colab o comunque hai wget installato sul tuo computer esegui pure la cella di codice qui sotto per scaricare il dataset.

In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-05-03 14:13:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-05-03 14:13:11--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  3.54MB/s    in 3m 4s   

2019-05-03 14:16:16 (4.46 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



Ed estraiamo il file.

In [10]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


Lo zip contiene 4 file differenti, ognuno dei quali con un numero di dimensioni differente, 50, 100, 200 e 300. Leggiamo la prima riga del file con 50 dimensioni per comprendere come questo è strutturato.

In [15]:
with open("glove.6B.50d.txt") as file:
  for line in file.readlines():
    print(line)
    break

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581



Come vedi ogni riga corrisponde all'embedding per una determianta parola, il primo elemento della riga è la parola, i restanti solo i valori degli embedding. Alla luce di ciò definiamo una funzione per caricare gli embedding all'iterno di un dizionario la cui chiave è la parola. Usiamo il file con 100 dimensioni.

In [16]:
def load_embedding(filename):
  
  with open(filename) as file:
    lines = file.readlines()

  embedding = dict()
  
  for line in lines:
    parts = line.split()
    embedding[parts[0]] = np.asarray(parts[1:])
    
  return embedding

raw_embedding = load_embedding('glove.6B.100d.txt')
raw_embedding["man"]

array(['0.37293', '0.38503', '0.71086', '-0.65911', '-0.0010128',
       '0.92715', '0.27615', '-0.056203', '-0.24294', '0.24632',
       '-0.18449', '0.31398', '0.48983', '0.09256', '0.32958', '0.15056',
       '0.57317', '-0.18529', '-0.52277', '0.46191', '0.92038',
       '0.031001', '-0.16246', '-0.40567', '0.78621', '0.57722',
       '-0.53501', '-0.68228', '0.16987', '0.3631', '-0.071773',
       '0.47233', '0.027806', '-0.14951', '0.17543', '-0.37573',
       '-0.78517', '0.58171', '0.86859', '0.031445', '-0.45897',
       '-0.040917', '0.95897', '-0.16975', '0.13045', '0.27434',
       '-0.069485', '0.022402', '0.24977', '-0.21536', '-0.32406',
       '-0.39867', '0.68613', '1.7923', '-0.37848', '-2.2477', '-0.77025',
       '0.46582', '1.2411', '0.57756', '0.41151', '0.84328', '-0.54259',
       '-0.16715', '0.73927', '-0.093477', '0.90278', '0.50889',
       '-0.50031', '0.26451', '0.15443', '-0.29432', '0.10906',
       '-0.26667', '0.35438', '0.049079', '0.18018', '-0.5859'

Adesso dobbiamo trasformare il dizionario in una matrice, in cui ogni riga rappresenta contiene la rappresentazione vettoriale della parola che si trova alla stessa posizione all'interno del vocabolario dell'intero corpus di testo. Keras ci mette a disposizione anche il vocabolario già creato per questo dataset. possiamo ottenerlo con la funzione *.get_word_index()*.

In [21]:
word_index = imdb.get_word_index()
word_index["man"]

129

Definiamo la funzione per creare la matrie ed utilizziamola.

In [40]:
def get_weight_matrix(embedding, word_index):
  
  vocab_size = len(word_index)
  
  weight_matrix = np.zeros((vocab_size, 100))

  for word, i in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
      weight_matrix[i] = vector
      
  return weight_matrix

embedding_vectors = get_weight_matrix(raw_embedding, word_index)
embedding_vectors.shape

(88584, 100)

# Creiamo la rete
Cominciamo creando lo strato di embedding della rete, i primi due parametri saranno il numero di parole e il numero di dimensioni, passiamo gli embeddings già calcolati all'interno del parametro *weights* e settiamo il parametro *trainable* a False per indicare di non modificare questo strato della rete durante l'addestramento.

In [0]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index), 100, weights=[embedding_vectors], trainable=False, input_length=maxlen)

Ora creiamo la nostra rete neurale aggiungendo al primo strato lo strato di embedding appena creato, usiamo la classe *Flatten* di Keras per spacchettare la matrice con il word embedding in un unico vettore che unisce le righe della matrice.

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

model = Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

Compiliamo il modello ed avviamo l'addestramento.

In [98]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 1s 41us/step - loss: 0.7187 - acc: 0.5211
Epoch 2/10
25000/25000 [==============================] - 0s 6us/step - loss: 0.6685 - acc: 0.5871
Epoch 3/10
25000/25000 [==============================] - 0s 6us/step - loss: 0.6483 - acc: 0.6206
Epoch 4/10
25000/25000 [==============================] - 0s 6us/step - loss: 0.6326 - acc: 0.6402
Epoch 5/10
25000/25000 [==============================] - 0s 6us/step - loss: 0.6229 - acc: 0.6519
Epoch 6/10
25000/25000 [==============================] - 0s 6us/step - loss: 0.6143 - acc: 0.6626
Epoch 7/10
25000/25000 [==============================] - 0s 6us/step - loss: 0.6088 - acc: 0.6671
Epoch 8/10
25000/25000 [==============================] - 0s 6us/step - loss: 0.6020 - acc: 0.6756
Epoch 9/10
25000/25000 [==============================] - 0s 6us/step - loss: 0.5993 - acc: 0.6758
Epoch 10/10
25000/25000 [==============================] - 0s 6us/step - loss: 0.5934 - acc: 0.6829


In [100]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 1s 39us/step


[0.6988005238342285, 0.57212]

Lo strato di embedding funziona correttamente, ma il risultato è piuttosto scarso, perché ? Perché utilizzare l'embedding senza tener conto della relazione temporale all'interno di una sequenza (come può essere una frase) è una cosa abbastanza inutile. Per tener conto delle informazioni sequenziali dobbiamo usare degli strati ricorrenti.

# Usiamo uno Strato Ricorrente
Le Reti Neurali Ricorrenti (RNN) saranno l'argomento della prossima sezione, qui vediamo semplicemente come varia il risultato utilizzandole insieme all'embedding.

In [0]:
from keras.layers import SimpleRNN

model = Sequential()
#model.add(Embedding(num_words, 50))
model.add(embedding_layer)
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

Come algoritmo di ottimizzazione usiamo l'RMSProp che dovrebbe portare a risultati migliori per le reti ricorrenti, siccome è più lento a convergere rispetto all'ADAM aumentiamo il numero di epoche dell'addestramento a 20.

In [107]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, epochs=20)

Epoch 1/20
25000/25000 [==============================] - 3s 101us/step - loss: 0.7050 - acc: 0.5268
Epoch 2/20
25000/25000 [==============================] - 1s 56us/step - loss: 0.6844 - acc: 0.5509
Epoch 3/20
25000/25000 [==============================] - 1s 57us/step - loss: 0.6764 - acc: 0.5638
Epoch 4/20
25000/25000 [==============================] - 1s 57us/step - loss: 0.6702 - acc: 0.5730
Epoch 5/20
25000/25000 [==============================] - 1s 58us/step - loss: 0.6648 - acc: 0.5824
Epoch 6/20
25000/25000 [==============================] - 1s 58us/step - loss: 0.6593 - acc: 0.5920
Epoch 7/20
25000/25000 [==============================] - 1s 57us/step - loss: 0.6548 - acc: 0.5954
Epoch 8/20
25000/25000 [==============================] - 1s 57us/step - loss: 0.6504 - acc: 0.6047
Epoch 9/20
25000/25000 [==============================] - 1s 51us/step - loss: 0.6471 - acc: 0.6115
Epoch 10/20
25000/25000 [==============================] - 1s 51us/step - loss: 0.6436 - acc: 0.616

In [108]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 13s 520us/step


[0.6410095851516724, 0.62128]